#  Vaex - 2D demo
The purpose of this notebook is to demonstrate
some of the 2D capabilities of VAEX. Plotting of data is performed with BQPlot
Jupyter widgets are enabled so the user can interactively play with the settings

In [1]:
import numpy as np
import ipywidgets as widgets
import bqplot as bq
import vaex as vx



### Create random data to plot

In [2]:

size = 10000
scale = 1.
scaleLocal = 20
np.random.seed(0)
x_data = np.arange(size)
y_data = np.cumsum(np.random.randn(size)  * scale) + np.random.randn(size) * scaleLocal

np.random.seed(1)
scaleLocal2 = 3
y_data2 = np.cumsum(np.random.randn(size)  * scale) + np.random.randn(size) * scaleLocal2


### Create inital figure and lines

In [3]:
x_sc = bq.LinearScale()
# x_sc.max = size * 1.3
y_sc = bq.LinearScale()
y_sc2 = bq.LinearScale()

ax_x = bq.Axis(label='X', scale=x_sc, grid_lines='solid')
ax_y = bq.Axis(label='Y', scale=y_sc, orientation='vertical', grid_lines='solid')
ax_y2 = bq.Axis(label='2nd Y', scale=y_sc2, orientation='vertical', side = 'right', visible = False,grid_lines='none')

line1 = bq.Lines(x=x_data, y=y_data, scales={'x': x_sc, 'y': y_sc} , colors=['blue'],display_legend = False, labels=['y1'],stroke_width = 1.0)

line2 = bq.Lines(x=x_data, y=y_data2, scales={'x': x_sc, 'y': y_sc} , colors=['darkgreen'],display_legend = False, labels=['y2'],stroke_width = 1.0)

margins = dict(top = 50, bottom=40, left=50, right=50)
fig = bq.Figure(marks = [line1,line2], axes=[ax_x, ax_y] , fig_margin = margins , animation_duration=1000)
fig.layout.width = '98%'
fig.layout.height = '400px' 
fig.title = 'Vaex demo'

figBinNum = widgets.IntText(
    value=500,
    description='Nº of bins',
    disabled=False
)
figBinNum.layout.width = '180px'


### Create Vaex object

In [4]:
dataVX = vx.from_arrays( x=x_data, y1=y_data, y2 = y_data2)

In [5]:
binbyChannelx = 'x'
binbyChannely = 'y1'

### Calculate initial binned line to plot

In [6]:
start = 500
end = 8000
samplePoints = (50,)
tExpression = "abs(" + str((start + end)/2) + " - " + binbyChannelx + ") < " + str((end-start)/2)
tExpressionAnd = "(" + binbyChannelx + " > " + str(start) + ") & (" + binbyChannelx + " < " + str(end) + ")"
dataVX.select(tExpression, name='xpos')
dataVX.select(tExpressionAnd, name='xposAnd')

x_Binned = dataVX.mean(binbyChannelx, binby=binbyChannelx, selection = 'xpos', shape=samplePoints, limits=[start,end])
y_Binned = dataVX.mean(binbyChannely, binby=binbyChannelx, selection = 'xpos', shape=samplePoints, limits=[start,end])

lineBinned = bq.Lines(x=x_Binned, y=y_Binned, scales={'x': x_sc, 'y': y_sc},display_legend = False, labels = ['y1 -- mean'])
lineBinned2ndY = bq.Lines(x=x_Binned, y=y_Binned, scales={'x': x_sc, 'y': y_sc2},display_legend = False, labels = ['y1 -- mean'])
fig.marks = [line1, line2, lineBinned]


### Create Jupyter widgets

In [7]:
UserEquation = widgets.Text(
    value='y1',
    placeholder='Type equation: e.g.  fuel * engspd,  or  where((fuel>80)&(engspd>2000) , tempegr_val, 0)',
    description='Equation1:',
    disabled=False
)
   
valid = widgets.Valid(
    value=True,
    description='',)
valid.layout.width = '20px'
equationHbox = widgets.HBox([UserEquation,valid])

UserEquation2 = widgets.Text(
    value='y2',
    placeholder='Type equation: e.g.  fuel * engspd,  or  where((fuel>80)&(engspd>2000) , tempegr_val, 0)',
    description='Equation2:',
    disabled=False
)
   
valid2 = widgets.Valid(
    value=True,
    description='',)
valid2.layout.width = '20px'

equationHbox2 = widgets.HBox([UserEquation2,valid2])

              
    
# -------------------------------
# User equation, time series channel


Userselect = widgets.Text(
    value='',
    placeholder='Type selection boolean equation',
    description='Selection:',
    disabled=False
)
   
validselect = widgets.Valid(
    value=True,
    description='',)
validselect.layout.width = '20px'
    
selectHbox = widgets.HBox([Userselect,validselect])
     
figBinNum = widgets.IntText(
    value=500,
    description='Nº of bins:',
    disabled=False
)
figBinNum.layout.width = '180px'

startx = widgets.FloatText(
    value=0,
    description='Start x:',
    disabled=False
)
startx.layout.width = '180px'

endx = widgets.FloatText(
    value=8000,
    description='End x:',
    disabled=False
)
endx.layout.width = '180px'

calcTypeWidget = widgets.SelectMultiple(
    options=['Mean', 'Max', 'Min', 'sum', 'Std. dev', '95%ile', 'median', 'variance', 'correlation' , 'covariance cov[x,y]'],
    value=['Mean'],
    #rows=10,
    description='Calc type:',
    disabled=False
)

buttonPlot = widgets.Button(description='Plot',
           layout=widgets.Layout(width='180px', height='25px'))

updatePlots_progressBar = widgets.IntProgress(
    value=0,
    min=0,
    max=10,
    step=1,
    description='',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal'
)
updatePlots_progressBar.layout.width = buttonPlot.layout.width
updatePlots_progressBar.layout.height = '15px'


helpLabelWidget1 = widgets.HTML(value="<b>The lines y1 and y2 are fixed.</b>")
helpLabelWidget = widgets.HTML(value="<b>Edit the settings and hit plot....</b>")


def on_buttonPlot_clicked(buttonPlot):       
    # clear_output() # clear old plots (and widgets which are reinserted in line below)
    updatePlot()

buttonPlot.on_click(on_buttonPlot_clicked)

buttonPlotBox = widgets.VBox([buttonPlot, updatePlots_progressBar])  
    
    
    

### Update plot function

In [8]:
class legendWidget(object):
    """A legend Widget using a horizontal bar chart
    
    marks: line marks from a bqplot figure.  
    
    These line marks must have legend labels 
    (in line mark, remove other legend by using this: display_legend = False)
    e.g. >>> legend = legendWidget(fig.marks) 
    
    BQplot module imported as bq (import bqplot as bq)

    """
    def __init__(self, marks):
        """Return a new Legend object."""
        y_ord = bq.OrdinalScale()
        x_sc = bq.LinearScale()
        
        legendLabels = []
        colours = []
        markLineNums = [] # record number of lines per mark
        for mark in marks:            
            legendLabels += mark.labels
            colours += mark.colors[:len(mark.labels)]
            markLineNums.append(len(mark.labels))  

        bar = bq.Bars(
            y=[1]*len(legendLabels) , # all bars have a amplitude of 1
            x=legendLabels, 
            scales={'y': x_sc, 'x': y_ord},
            colors=colours ,
            padding = 0.6,
            orientation='horizontal',
            stroke = 'white'  #remove the black border around the bar
            )
        
        ax_y = bq.Axis(scale=y_ord, orientation="vertical")
        ax_x = bq.Axis(scale=x_sc)
        ax_x.visible = False
        margin = dict(top=40, bottom=0, left=110, right=5)
        barFig = bq.Figure(marks=[bar], axes=[ax_y, ax_x], fig_margin=margin)
        
        # Variable height depending on number of bars in legend
        barFig.layout.height = str(45 + 20 * len(legendLabels)) + 'px'
        barFig.layout.width = '170px'

        barFig.min_aspect_ratio = 0.000000000001 # effectively remove aspect ratio constraint
        barFig.max_aspect_ratio = 999999999999999 # effectively remove aspect ratio constraint
        barFig.background_style = {'fill': 'White'}   
                    
        self.fig = barFig
        self.bar = bar
        self.colours = colours
        self.markLineNums = markLineNums
        

 
    
        

In [9]:
legend = legendWidget(fig.marks)        

def changeOpacity(self, target):
    """Enable legend interactivity. 
    Use in conjunction with class legendWidget(object) 
    Click on legend bar to toggle opacity of all other lines
    
    """
    
    # I'm not sure how to pass in the line chart and legend widgets from on_element_click(). 
    # Need to explicity define them below.
    lineFig = fig  # set lineFig to name of line chart figure
    legendFig = legend  # set legendFig to name of new legend widget
    
    opacity = 0.1   # set opacity of non selected lines here
    sigNum = target['data']['index']
    bar = self
    if bar.opacities == [] or bar.opacities[sigNum] == opacity:
        bar.opacities=[opacity]*sigNum + [1] + [opacity]*(len(bar.x) - sigNum - 1)        
        
        # Some marks in line plot have more than 1 line.  
        currentLineNum=0
        for markNum,markLineNum in enumerate(legendFig.markLineNums):
            lineFig.marks[markNum].opacities = bar.opacities[currentLineNum:currentLineNum + markLineNum ]
            currentLineNum+=markLineNum
    else:
        bar.opacities = []
        for mark in lineFig.marks:
#         for mark in self:
            mark.opacities = []
   
legend.fig.marks[0].on_element_click(changeOpacity)
legend.fig

In [10]:
binbyChannelx = 'x'

def updatePlot():
    binbyChannelys =  []
    if UserEquation.value !='':
        binbyChannelys.append(UserEquation.value)
    if UserEquation2.value !='':
        binbyChannelys.append(UserEquation2.value)
        
    updatePlots_progressBar.max = len(calcTypeWidget.value) * len(binbyChannelys)
    
    start = startx.value
    end = endx.value
    samplePoints = figBinNum.value
    tExpressionAnd = "(" + binbyChannelx + " > " + str(start) + ") & (" + binbyChannelx + " < " + str(end) + ")"
    dataVX.select(tExpressionAnd)

    x_Binned = dataVX.mean(binbyChannelx, binby=binbyChannelx, selection = True, shape=(samplePoints,), limits=[start,end])
    print('x_Binned = dataVX.mean(' + binbyChannelx + ' , binby=' + binbyChannelx + ', selection = True, shape = (' + str(samplePoints) + ',) , limits=[' + str(start) +', ' + str(end) + '])')

    print('dataVX.select(' + tExpressionAnd + ')')

    if Userselect.value!= '':
        print('dataVX.select(' + Userselect.value + ', mode = \'and\')')
        dataVX.select(Userselect.value, mode = 'and')    
#         selectionLabel = 'Selection: ' + Userselect.value
    
    y_Binned=[]
    y_Binned2=[]
    labels = []
    labels2 = []
#     print('y_Binned=[]')
    for calcType in calcTypeWidget.value:
        for binbyChannely in binbyChannelys:
            updatePlots_progressBar.value +=1                
            if calcType == 'Mean':
                y_Binned.append(dataVX.mean(binbyChannely, binby=binbyChannelx, selection = True, shape=(samplePoints,), limits=[start,end]) )
                vaexCommand = 'mean'
                labels.append(binbyChannely + '--' + calcType )
                print('dataVX.' + vaexCommand + '("' + binbyChannely + '", binby="' + binbyChannelx + '", selection = True, shape = (' + str(samplePoints) + ',) , limits=[' + str(start) +', ' + str(end) + '])')
            elif calcType == 'Max':
                y_Binned.append(dataVX.max(binbyChannely, binby=binbyChannelx, selection = True, shape=(samplePoints,), limits=[start,end]) )
                vaexCommand = 'max'
                labels.append(binbyChannely + '--' + calcType )
                print('dataVX.' + vaexCommand + '("' + binbyChannely + '", binby="' + binbyChannelx + '", selection = True, shape = (' + str(samplePoints) + ',) , limits=[' + str(start) +', ' + str(end) + '])')
            elif calcType == 'Min':
                y_Binned.append(dataVX.min(binbyChannely, binby=binbyChannelx, selection = True, shape=(samplePoints,), limits=[start,end]) )
                vaexCommand = 'min'
                labels.append(binbyChannely + '--' + calcType )
                print('dataVX.' + vaexCommand + '("' + binbyChannely + '", binby="' + binbyChannelx + '", selection = True, shape = (' + str(samplePoints) + ',) , limits=[' + str(start) +', ' + str(end) + '])')
            elif calcType == 'sum':
                y_Binned2.append(dataVX.sum(binbyChannely, binby=binbyChannelx, selection = True, shape=(samplePoints,), limits=[start,end]) )
                vaexCommand = 'sum'
                labels2.append(binbyChannely + '--' + calcType )
                print('dataVX.' + vaexCommand + '("' + binbyChannely + '", binby="' + binbyChannelx + '", selection = True, shape = (' + str(samplePoints) + ',) , limits=[' + str(start) +', ' + str(end) + '])')
            elif calcType == 'Std. dev':
                y_Binned2.append(dataVX.std(binbyChannely, binby=binbyChannelx, selection = True, shape=(samplePoints,), limits=[start,end]) )
                vaexCommand = 'std'
                labels2.append(binbyChannely + '--' + calcType )
                print('dataVX.' + vaexCommand + '("' + binbyChannely + '", binby="' + binbyChannelx + '", selection = True, shape = (' + str(samplePoints) + ',) , limits=[' + str(start) +', ' + str(end) + '])')
            elif calcType == 'variance':
                y_Binned2.append(dataVX.var(binbyChannely, binby=binbyChannelx, selection = True, shape=(samplePoints,), limits=[start,end]) )
                vaexCommand = 'var'
                labels2.append(binbyChannely + '--' + calcType )
                print('dataVX.' + vaexCommand + '("' + binbyChannely + '", binby="' + binbyChannelx + '", selection = True, shape = (' + str(samplePoints) + ',) , limits=[' + str(start) +', ' + str(end) + '])')
            elif calcType == '95%ile':
                y_Binned.append(dataVX.percentile_approx(binbyChannely, binby=binbyChannelx, selection = True, shape=(samplePoints,), limits=[start,end]) )
                vaexCommand = 'percentile_approx'
                labels.append(binbyChannely + '--' + calcType )
                print('dataVX.' + vaexCommand + '("' + binbyChannely + '", percentage=95.0 , binby="' + binbyChannelx + '", selection = True, shape = (' + str(samplePoints) + ',) , limits=[' + str(start) +', ' + str(end) + '])')
            elif calcType == 'median':
                y_Binned.append(dataVX.median_approx(binbyChannely, binby=binbyChannelx, selection = True, shape=(samplePoints,), limits=[start,end]) )
                vaexCommand = 'median_approx'
                labels.append(binbyChannely + '--' + calcType )
                print('dataVX.' + vaexCommand + '("' + binbyChannely + '", percentage=50.0 , binby="' + binbyChannelx + '", selection = True, shape = (' + str(samplePoints) + ',) , limits=[' + str(start) +', ' + str(end) + '])')
            else:
                break
        if len(binbyChannelys) ==2:
            if calcType == 'correlation':
                y_Binned2.append(dataVX.correlation(binbyChannelys[0], binbyChannelys[1],binby=[binbyChannelx], selection = True, shape=(samplePoints,), limits=[start,end]) )
                vaexCommand = 'correlation'
                labels2.append(binbyChannely[0] + ',' +  binbyChannely[1]+ '--' + calcType )
                print('dataVX.' + vaexCommand + '("' + binbyChannelys[0] + '","' + binbyChannelys[1] + '", binby=["' + binbyChannelx + '"], selection = True, shape = (' + str(samplePoints) + ',) , limits=[' + str(start) +', ' + str(end) + '])')
            elif calcType == 'covariance cov[x,y]':
                y_Binned2.append(dataVX.covar(binbyChannelys[0], binbyChannelys[1],binby=[binbyChannelx], selection = True, shape=(samplePoints,), limits=[start,end]) )
                vaexCommand = 'covar'
                labels2.append(binbyChannely[0] + ',' + binbyChannely[1]+ '--' + calcType )
                print('dataVX.' + vaexCommand + '("' + binbyChannelys[0] + '","' + binbyChannelys[1] + '", binby=["' + binbyChannelx + '"], selection = True, shape = (' + str(samplePoints) + ',) , limits=[' + str(start) +', ' + str(end) + '])')
#             elif calcType == 'variance':
#                 y_Binned2.append(dataVX.var(binbyChannelys[0], binbyChannelys[1],binby=[binbyChannelx], selection = True, shape=(samplePoints,), limits=[start,end]) )
#                 vaexCommand = 'var'
#                 print('dataVX.' + vaexCommand + '("' + binbyChannelys[0] + ',' + binbyChannelys[1] + '", binby=[' + binbyChannelx + '], selection = True, shape = (' + str(samplePoints) + ',) , limits=[' + str(start) +', ' + str(end) + '])')
        else:
            if calcType in ['correlation' , 'covariance','covariance cov[x,y]']:
                print('Please create 2 equations to compute ' + calcType)

    lineBinned.x = x_Binned
    lineBinned.y = y_Binned
    lineBinned.labels = labels
    if y_Binned2 !=[]:
        lineBinned2ndY.x = x_Binned
        lineBinned2ndY.y = y_Binned2
        lineBinned2ndY.labels = labels2
        ax_y2.visible =True
        fig.marks =  [line1, line2, lineBinned, lineBinned2ndY]
        fig.axes =[ax_x, ax_y, ax_y2]
        
    else:
        ax_y2.visible =False
        fig.marks =  [line1, line2, lineBinned]
        fig.axes =[ax_x, ax_y]
        
#     y_sc2 = bq.LinearScale()
#     ax_y2 = bq.Axis(label='2nd Y', scale=y_sc2, orientation='vertical', side = 'right', visible = False)
    
    legendNew  = legendWidget(fig.marks)
    legendFig = legendNew.fig
    legend.fig.marks = legendFig.marks
    legend.markLineNums = legendNew.markLineNums
    legend.fig.axes = legendFig.axes
    legend.fig.layout.height = str(40 + 20 * sum(legendNew.markLineNums)) + 'px'
    legend.fig.marks[0].on_element_click(changeOpacity)
    updatePlots_progressBar.value=0  

    print('------------------------------------------------------------------------')

In [11]:

fig_legend =  widgets.HBox([fig,legend.fig])
lhs = widgets.VBox([helpLabelWidget1,helpLabelWidget,buttonPlotBox])
rhs = widgets.VBox([figBinNum,widgets.HBox([startx,endx]), equationHbox,equationHbox2,selectHbox,calcTypeWidget])
bottom = widgets.HBox([lhs,rhs])

widgets.VBox([fig_legend,bottom])

VBox(children=(HBox(children=(Figure(animation_duration=1000, axes=[Axis(label='X', scale=LinearScale()), Axis(label='Y', orientation='vertical', scale=LinearScale())], fig_margin={'top': 50, 'bottom': 40, 'left': 50, 'right': 50}, layout=Layout(height='400px', min_width='125px', width='98%'), marks=[Lines(colors=['blue'], interactions={'hover': 'tooltip'}, labels=['y1'], scales={'x': LinearScale(), 'y': LinearScale()}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, stroke_width=1.0, tooltip_style={'opacity': 0.9}, x=array([   0,    1,    2, ..., 9997, 9998, 9999]), y=array([  -2.2782882 ,  -14.50041046,   37.81495249, ..., -186.74757737,
       -206.81393825, -190.86777043])), Lines(colors=['darkgreen'], interactions={'hover': 'tooltip'}, labels=['y2'], scales={'x': LinearScale(), 'y': LinearScale()}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, stroke_width=1.0, tooltip_style={'opacity': 0.9}, x=array([   0,    1,    2, ..., 9997, 9998, 9999]), y=array([   1.25692364,    1.69709841,   -0.57249819, ...,  106.4524345 ,
        101.81279353,   97.42777806])), Lines(colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'], interactions={'hover': 'tooltip'}, labels=['y1 -- mean'], scales={'x': LinearScale(), 'y': LinearScale()}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip_style={'opacity': 0.9}, x=array([  575. ,   724.5,   874.5,  1024.5,  1174.5,  1324.5,  1474.5,
        1624.5,  1774.5,  1924.5,  2074.5,  2224.5,  2374.5,  2524.5,
        2674.5,  2824.5,  2974.5,  3124.5,  3274.5,  3424.5,  3574.5,
        3724.5,  3874.5,  4024.5,  4174.5,  4324.5,  4474.5,  4624.5,
        4774.5,  4924.5,  5074.5,  5224.5,  5374.5,  5524.5,  5674.5,
        5824.5,  5974.5,  6124.5,  6274.5,  6424.5,  6574.5,  6724.5,
        6874.5,  7024.5,  7174.5,  7324.5,  7474.5,  7624.5,  7774.5,
        7924.5]), y=array([ -28.71770235,  -44.81723706,  -57.63340053,  -44.28288604,
        -33.24696906,  -32.02187818,  -31.71313552,  -25.20421347,
        -26.75606596,  -32.38881645,  -28.20298608,  -44.0675195 ,
        -49.4973406 ,  -51.2358642 ,  -58.28172053,  -63.59702301,
        -79.33253318,  -94.26311393, -101.89867384, -104.46517768,
        -92.28603283,  -92.53081774,  -98.46231637,  -99.77228422,
       -113.74517421, -111.31417219, -103.48450689,  -96.90158546,
        -99.49096564,  -82.56879952,  -81.49509138,  -91.58382754,
        -85.40262056,  -90.09065324,  -90.77984115,  -81.78800491,
        -87.24448112,  -86.10147012,  -79.45352057,  -88.80388338,
        -96.03586617,  -94.35885361,  -83.53224163,  -80.46050655,
        -98.70004682, -104.84187477,  -97.64325881, -105.93949825,
        -94.02696374,  -98.91303508]))], scale_x=LinearScale(allow_padding=False, max=1.0, min=0.0), scale_y=LinearScale(allow_padding=False, max=1.0, min=0.0), title='Vaex demo'), Figure(axes=[Axis(orientation='vertical', scale=OrdinalScale()), Axis(scale=LinearScale(), visible=False)], background_style={'fill': 'White'}, fig_margin={'top': 40, 'bottom': 0, 'left': 110, 'right': 5}, layout=Layout(height='105px', min_width='125px', width='170px'), marks=[Bars(colors=['blue', 'darkgreen', '#1f77b4'], interactions={'hover': 'tooltip'}, orientation='horizontal', padding=0.6, scales={'y': LinearScale(), 'x': OrdinalScale()}, scales_metadata={'x': {'orientation': 'vertical', 'dimension': 'x'}, 'y': {'orientation': 'horizontal', 'dimension': 'y'}}, stroke='white', tooltip_style={'opacity': 0.9}, x=array(['y1', 'y2', 'y1 -- mean'],
      dtype='<U10'), y=array([1, 1, 1]))], max_aspect_ratio=999999999999999.0, min_aspect_ratio=1e-12, scale_x=LinearScale(allow_

x_Binned = dataVX.mean(x , binby=x, selection = True, shape = (500,) , limits=[0.0, 8000.0])
dataVX.select((x > 0.0) & (x < 8000.0))
dataVX.mean("y1", binby="x", selection = True, shape = (500,) , limits=[0.0, 8000.0])
dataVX.mean("y2", binby="x", selection = True, shape = (500,) , limits=[0.0, 8000.0])
dataVX.max("y1", binby="x", selection = True, shape = (500,) , limits=[0.0, 8000.0])
dataVX.max("y2", binby="x", selection = True, shape = (500,) , limits=[0.0, 8000.0])
dataVX.min("y1", binby="x", selection = True, shape = (500,) , limits=[0.0, 8000.0])
dataVX.min("y2", binby="x", selection = True, shape = (500,) , limits=[0.0, 8000.0])
dataVX.sum("y1", binby="x", selection = True, shape = (500,) , limits=[0.0, 8000.0])
dataVX.sum("y2", binby="x", selection = True, shape = (500,) , limits=[0.0, 8000.0])
dataVX.std("y1", binby="x", selection = True, shape = (500,) , limits=[0.0, 8000.0])
dataVX.std("y2", binby="x", selection = True, shape = (500,) , limits=[0.0, 8000.0])
---------------

In [12]:
# Only include in calculation when y2 is greater than 50
Userselect.value = 'y2 < 50'
updatePlot()

x_Binned = dataVX.mean(x , binby=x, selection = True, shape = (500,) , limits=[0.0, 8000.0])
dataVX.select((x > 0.0) & (x < 8000.0))
dataVX.select(y2 < 50, mode = 'and')
dataVX.mean("y1", binby="x", selection = True, shape = (500,) , limits=[0.0, 8000.0])
dataVX.mean("y2", binby="x", selection = True, shape = (500,) , limits=[0.0, 8000.0])
------------------------------------------------------------------------


In [13]:
UserEquation.value = 'y1*y2 / 10'
updatePlot()

x_Binned = dataVX.mean(x , binby=x, selection = True, shape = (500,) , limits=[0.0, 8000.0])
dataVX.select((x > 0.0) & (x < 8000.0))
dataVX.select(y2 < 50, mode = 'and')
dataVX.mean("y1*y2 / 10", binby="x", selection = True, shape = (500,) , limits=[0.0, 8000.0])
dataVX.mean("y2", binby="x", selection = True, shape = (500,) , limits=[0.0, 8000.0])
------------------------------------------------------------------------


In [14]:
# Using the numexpr where function, cap the minimum to -1000
UserEquation.value = 'where(y1*y2 / 10 < -1000 , -1000 ,y1*y2 / 10) '
updatePlot()

x_Binned = dataVX.mean(x , binby=x, selection = True, shape = (500,) , limits=[0.0, 8000.0])
dataVX.select((x > 0.0) & (x < 8000.0))
dataVX.select(y2 < 50, mode = 'and')
dataVX.mean("where(y1*y2 / 10 < -1000 , -1000 ,y1*y2 / 10) ", binby="x", selection = True, shape = (500,) , limits=[0.0, 8000.0])
dataVX.mean("y2", binby="x", selection = True, shape = (500,) , limits=[0.0, 8000.0])
------------------------------------------------------------------------


In [15]:
Userselect.value = ''
UserEquation.value = 'y1'
updatePlot()

x_Binned = dataVX.mean(x , binby=x, selection = True, shape = (500,) , limits=[0.0, 8000.0])
dataVX.select((x > 0.0) & (x < 8000.0))
dataVX.mean("y1", binby="x", selection = True, shape = (500,) , limits=[0.0, 8000.0])
dataVX.mean("y2", binby="x", selection = True, shape = (500,) , limits=[0.0, 8000.0])
------------------------------------------------------------------------
